# RChain Python gRPC client 

- This notebook is available at http://j.mp/rchain-py-grpc-walk-through

- github https://github.com/proof-media/rchain-grpc
- pypi https://pypi.org/user/proof-media/

## Introduction

### How to reproduce this demo

- Interactive locally
- Read-only mode on the web

#### Locally on your workstation 
with Docker containers

```bash
git clone https://github.com/proof-media/rchain-notebook.git
cd rchain-notebook
docker-compose up
```

Then open http://localhost:8888

In [3]:
# password is set in the `.env` file
# and shared to notebook container
%env NOTEBOOK_PASSWORD

'prooflikesrchainalot'

#### Read only mode

Through https://nbviewer.jupyter.org/

- [link to notebook](https://github.com/proof-media/rchain-notebook/blob/testing/name-registry/notebooks/walk-through.ipynb)
- [link to slides](https://nbviewer.jupyter.org/format/slides/github/proof-media/rchain-notebook/blob/testing%2Fname-registry/notebooks/walk-through.ipynb#/)

### Proof platform

![proof image](https://i0.wp.com/proofmedia.io/wp-content/uploads/2018/06/Default-Logo-copyxhdpi.png?resize=246%2C159&ssl=1)

**Because Truth Matters**

https://proofmedia.io/

> We are building a platform which leverages the wisdom of the crowds to establish truth.

- people vote in our platform on articles for being Mostly True/False
- users bet tokens to rank their conviction 
- voting closes when we reach some parameters that are proven to make "the crowd wise"
- tallying the vote will find out what the *majority* says
- we use the blockchain to lock the tokens and provide cryptographic proofs that votes weren't tampered

Rchain is needed to *register* tokens/hashes computed 

when each user votes

### Credits

Our backend is based on `Python 3.7+` and `Django framework`

- inspired by the official rchain/python snippet 

https://github.com/rchain/rchain/tree/dev/node-client


- developed mainly by Mateusz (a.k.a. `beetleman`)

https://github.com/proof-media/rchain-grpc/graphs/contributors

- I'm Paolo - CTO @Proof
- Pythonista 
- Python teacher using Jupyter Notebooks

### Roadmap and status

In [2]:
! pip search rchain | grep -i grpc

rchain-grpc (0.0.10)  - python client to rchain gRPC


* Up to version `0.0.10` -> code developed tested with `RNode 0.6.x`
* Upcoming `0.0.11` release -> compatible with `RNode 0.7.x`

- Current [update to 0.7 PR](https://github.com/proof-media/rchain-grpc/pull/10) is blocked by a [protobuf issue](https://github.com/protocolbuffers/protobuf/issues/5272)

- Eval/Repl [not working](https://github.com/proof-media/rchain-grpc/issues/12) for gRPC missing definition


- Short-term goal

move the repo into `rchain` GitHub space, see [issue](https://github.com/proof-media/rchain-grpc/issues/8)

(after those previous problems are solved)

## Using the code

### Connecting

With current `docker-compose` for the notebook you are running a RNode *standalone* instance

In [7]:
# we have the OS NODE_NAME variable holding the other container name
RNODE_HOST = %env NODE_NAME

In [5]:
! apk add curl

fetch http://dl-cdn.alpinelinux.org/alpine/v3.8/main/x86_64/APKINDEX.tar.gz
fetch http://dl-cdn.alpinelinux.org/alpine/v3.8/community/x86_64/APKINDEX.tar.gz
(1/1) Installing curl (7.61.1-r0)
7  0%                                                                           87100% ██████████████████████████████████████████████████████████████████████████8Executing busybox-1.28.4-r1.trigger
OK: 36 MiB in 48 packages


In [8]:
! curl $RNODE_HOST:40403/version

/bin/sh: curl: not found


Now we can open a channel through `gRPC` against the RChain node 

using the Python library!

NOTE: in running docker image the package is already installed,

while normally you would do:
```bash
pip install rchain-grpc
```

In [9]:
from rchain_grpc import casper

connection = casper.create_connection(host=RNODE_HOST)
connection

### Write Rholang code

In a notebook you can write a file like this

In [3]:
%%file hello_world.rho

new print(`rho:io:stdout`) in {
    print!("Hello World!")
}


Writing hello_world.rho


Then edit it inside the notebook server,
e.g. <a href="http://localhost:8888/edit/hello_world.rho" target="_blank">link to file</a>

In [58]:
# then read the file to use the code in python
with open('hello_world.rho') as fh:
    rholang_code = fh.read()

In [10]:
# alternative: write directly the code with triple quoted strings
rholang_code = """
new print(`rho:io:stdout`) in {
    print!("Hello World!")
}
"""

In [11]:
print(rholang_code)


new print(`rho:io:stdout`) in {
    print!("Hello World!")
}



### Deploy and propose

In [12]:
casper.deploy(connection, rholang_code)

{'success': True, 'message': 'Success!'}

In [13]:
casper.propose(connection)

{'success': True, 'message': 'Success! Block 4b15fa7621... created and added.'}

Now check logs of the rnode to see if we have `Hello World!` string

### Play with blocks

In [14]:
output = casper.get_blocks(connection, depth=1)

# NOTE: protobuf outputs are always converted into Python dictionaries by our library
output

[{'blockHash': '4b15fa762195810bcc89a8db188927a4f1517f9a0d175f06b468908fb4bde506',
  'blockSize': '1340',
  'blockNumber': 1,
  'deployCount': 1,
  'tupleSpaceHash': '781bbcd09fa259caa17bab1c2d6b339c5bee99576bc04c8c012e5c06a9a0316c',
  'timestamp': 1540215055121,
  'faultTolerance': -1.0,
  'mainParentHash': '8d08dd96ef1152951b80b551fb1bfb928f0a5e46742038da52c2be39e5f16d26',
  'parentsHashList': ['8d08dd96ef1152951b80b551fb1bfb928f0a5e46742038da52c2be39e5f16d26'],
  'sender': 'eabe5a1a0750d2a8745709bb0bdb24f63c6a8ac3a887b9bed40b34b0598ddf08'}]

In [15]:
block_hash = output.pop().get('blockHash')
block_hash

'4b15fa762195810bcc89a8db188927a4f1517f9a0d175f06b468908fb4bde506'

In [16]:
block = casper.get_block(connection, block_hash=block_hash)
block

{'blockHash': '4b15fa762195810bcc89a8db188927a4f1517f9a0d175f06b468908fb4bde506',
 'blockSize': '1340',
 'blockNumber': 1,
 'deployCount': 1,
 'tupleSpaceHash': '781bbcd09fa259caa17bab1c2d6b339c5bee99576bc04c8c012e5c06a9a0316c',
 'tupleSpaceDump': '@{Unforgeable(0xa4fd447dedfc960485983ee817632cf36d79f45fd1796019edfb4a84a81d1697)}!({}) |\n@{"proofOfStake"}!(Unforgeable(0xc02c72fad36bc255f9f93a5b569d4d2bd502bed3fe6ed37534a544113c1e4e3f)) |\n@{Unforgeable(0x651923755db27e1ed7e69bd7b8576330a2d20c7b16253b46e71b5c1386b91e2d)}!(1222) |\n@{Unforgeable(0x51ff32c8ed4b7b774ab5fe0c5e4f7e3b5bd4bdf31903ddf8e5ca65c6aa4070b5)}!({5122eac8a5c99b7da8f6d40d5f3269c6844dabde0418749ece2aeedc64c27703 : (288, "secp256k1Verify", Nil, 1), 5a3fff1ed432237e779fc6aa20d7549d043cc0dd92180ebee0346229598870f8 : (228, "secp256k1Verify", Nil, 2), d6b0c603a51fbcd56db1606a0591310876f93bdbf2309c7bdbc277a080368990 : (188, "secp256k1Verify", Nil, 4), a3158195b3ca6dfb88b8f0cb0788b01f25ea11421104df071bd043ca27f2def4 : (202, "se

In [17]:
block.get('blockNumber')

1

### Context Manager

The connection provided by the library can be used in a `with` python statement

In [18]:
# all previous operations in one context

with casper.create_connection(host=RNODE_HOST) as connection:

    # deploy / propose
    casper.deploy(connection, rholang_code)
    print(casper.propose(connection))
    
    # handle output
    output = casper.get_blocks(connection, depth=1)
    block_hash = output.pop().get('blockHash')
    block = casper.get_block(connection, block_hash=block_hash)
    print(f"Current block number is {block.get('blockNumber')}\nwith hash {block_hash}")
    
# connection here is closed

{'success': True, 'message': 'Success! Block 40d0252c21... created and added.'}
Current block number is 2
with hash 40d0252c212fb57e09a812b8356d06926ead79becc86e4c4e02c6f3d543a275c


### Interact with channels

Now we'll take advantage of the `listenForDataAtName` functionality inside the RChain gRPC definition.

We can specify a channel to receive the results of the new block proposed.

In [20]:
output_placeholder = "your_channel_name"

In [21]:
rholang_code = f"""
{output_placeholder}!("bar")
""" 

In [22]:
print(rholang_code)

# your_channel_name will be replaced with @"RANDOMNAME"


your_channel_name!("bar")



In [43]:
with casper.create_connection(host=RNODE_HOST) as connection:
    block = casper.run_and_get_value_from(
        connection=connection,
        term=rholang_code, 
        # this name gets replaced with a channel name
        output_placeholder=output_placeholder
    )

In [44]:
# what's used for replacing the placeholder
import secrets
ack_name = f'ack_{secrets.token_hex(30)}'
ack_name

'ack_b2859ffa8914757d71c63ff685b5b880f36f0a1e69952a7d1c2f23b44008'

In [45]:
block

{'status': 'Success',
 'blockResults': [{'postBlockData': [['bar']],
   'block': {'blockHash': '14e19feeba18e8840bd85e2dda5d7bf93066c10e228ebbffe0d9ad23f541d276',
    'blockSize': '1144',
    'blockNumber': 6,
    'deployCount': 1,
    'tupleSpaceHash': 'd7184933b08e4d3e23afc3451de901a4c6fe0507cff15f7349ecf6cf051a50a8',
    'timestamp': 1540215534547,
    'faultTolerance': -1.0,
    'mainParentHash': '05ff25f824832ff5219e01be5eb4adeb05976ad5612d7b74981e2002915e708d',
    'parentsHashList': ['05ff25f824832ff5219e01be5eb4adeb05976ad5612d7b74981e2002915e708d'],
    'sender': 'eabe5a1a0750d2a8745709bb0bdb24f63c6a8ac3a887b9bed40b34b0598ddf08'}}],
 'length': 1}

In [46]:
results = block.get('blockResults').pop()

for message in results.get('postBlockData'):
    print("Received: ", message.pop())

Received:  bar


### Name registry

`NEW!`

(see [link to specs](https://rchain.atlassian.net/wiki/spaces/RHOL/pages/511541342/Name+registry+specification))

Using the techniques tested so far we can now:

- write a contract with `unforgeable name`
- register its name to Name Registry (introduced in 0.7) 
- and get the id for lookup

so - all programmatically from Python - 

we can (almost) call a contract in a secure way.

#### 1. Register a contract

In [47]:
ack_name = 'placeholder_something'

In [48]:
rholang_code = """
new newName, ack, register(`rho:registry:insertArbitrary`), stdout(`rho:io:stdout`) in
{
  register!(bundle+{*newName}, *ack)
  |
  contract newName(@msg) = {
    stdout!("Contract called with message: " ++ msg)
  }
  |
  for (@msg <- ack) {
    %s!(["From registry: ", msg])
  }
}
""" % ack_name

In [49]:
print(rholang_code)


new newName, ack, register(`rho:registry:insertArbitrary`), stdout(`rho:io:stdout`) in
{
  register!(bundle+{*newName}, *ack)
  |
  contract newName(@msg) = {
    stdout!("Contract called with message: " ++ msg)
  }
  |
  for (@msg <- ack) {
    placeholder_something!(["From registry: ", msg])
  }
}



In [53]:
# run and get the value in the block output
with casper.create_connection(host=RNODE_HOST) as connection:
    block = casper.run_and_get_value_from(connection, rholang_code, ack_name)
    
from pprint import pprint
pprint(block)

{'blockResults': [{'block': {'blockHash': '8d40c550e9cd169c4bf80eb2d598835aed674f72becf933571d2406ea835a25e',
                             'blockNumber': 9,
                             'blockSize': '3185',
                             'deployCount': 1,
                             'faultTolerance': -1.0,
                             'mainParentHash': '40472937049565f99775e329dfdb334b5af4a362baf38138ea5b3cd836f2c3c8',
                             'parentsHashList': ['40472937049565f99775e329dfdb334b5af4a362baf38138ea5b3cd836f2c3c8'],
                             'sender': 'eabe5a1a0750d2a8745709bb0bdb24f63c6a8ac3a887b9bed40b34b0598ddf08',
                             'timestamp': 1540215840006,
                             'tupleSpaceHash': '2c0846b670edfd62a536b5e6cfa7307037c5144105f7676075f755f0a844b95c'},
                   'postBlockData': [[{'ps': [['From registry: '],
                                              ['rho:id:c1wqhooe5f9f8st4qe69zs7t5jrj8zszbx4zd51qu7pi5sdc6hh1iw']]}

#### 2. WIP: get ID inside the code

We are looking to find a better way to handle the "post block data" 

In [54]:
post_block_data = block.get('blockResults').pop().get('postBlockData')

In [55]:
post_block_data

[[{'ps': [['From registry: '],
    ['rho:id:c1wqhooe5f9f8st4qe69zs7t5jrj8zszbx4zd51qu7pi5sdc6hh1iw']]}]]

In [56]:
list(post_block_data[0][0].values())[0][1][0]

'rho:id:c1wqhooe5f9f8st4qe69zs7t5jrj8zszbx4zd51qu7pi5sdc6hh1iw'

In [59]:
# alternative with json/regular expression

import re
import json

post_block_str = json.dumps(post_block_data)

match = re.search(r"rho:id:[^\"]+",  post_block_str)
registry_id = match.group()

In [60]:
registry_id

'rho:id:c1wqhooe5f9f8st4qe69zs7t5jrj8zszbx4zd51qu7pi5sdc6hh1iw'

#### 3. Lookup and call the contract

In [63]:
rholang_code = """
new return, lookup(`rho:registry:lookup`), stdout(`rho:io:stdout`) in
{
  lookup!(`%s`, *return) |
  for (myContract <- return) {
    myContract!("Proof test")
  }
}
""" % registry_id
print(rholang_code)


new return, lookup(`rho:registry:lookup`), stdout(`rho:io:stdout`) in
{
  lookup!(`rho:id:c1wqhooe5f9f8st4qe69zs7t5jrj8zszbx4zd51qu7pi5sdc6hh1iw`, *return) |
  for (myContract <- return) {
    myContract!("Proof test")
  }
}



In [64]:
with casper.create_connection(host=RNODE_HOST) as connection:
    print(casper.deploy(connection, rholang_code))
    print(casper.propose(connection))

{'success': True, 'message': 'Success!'}
{'success': True, 'message': 'Success! Block 06319c14c3... created and added.'}


go check the logs!

# THE END